In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
print(torch.cuda.is_available())

True


# Linear Layer ([Documentation](http://pytorch.org/docs/stable/generated/torch.nn.Linear.html))
``torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)``


Let us start with a single neuron having 10 inputs and just 1 output, but no bias

## Single Neuron

Let us start with a single neuron having 10 input features and just 1 output feature

In [7]:
single_neuron = nn.Linear(10, 1, bias=False)


PyTorch automatically ininitializes the weights of all layers, we can look at those weights by calling

In [8]:
print(single_neuron.weight.shape)
print(single_neuron.bias)

torch.Size([1, 10])
None


In [9]:
single_neuron.weight

Parameter containing:
tensor([[-0.0337,  0.3056,  0.2598, -0.0097,  0.2660, -0.3115, -0.0368,  0.1974,
          0.1745, -0.3011]], requires_grad=True)

An All-Zero Output always evaluates to 0 when there is no bias. Let us verify this by creating a suitable input tensor of ``(b, input_features) == (1, 10)`` with ``b`` being the batch size, ``input_features`` being the number of input features.

In [10]:
input_0 = torch.zeros((1, 10))
print(input_0.shape)
input_1 = torch.ones((1, 10))
print(input_1)
input_1000 = 1000 * torch.ones((1, 10))
print(input_1000)

torch.Size([1, 10])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.]])


In [11]:
print(single_neuron(input_0))
print(single_neuron(input_1))
print(single_neuron(input_1000))

tensor([[0.]], grad_fn=<MmBackward0>)
tensor([[0.5105]], grad_fn=<MmBackward0>)
tensor([[510.5245]], grad_fn=<MmBackward0>)


In [12]:
# check the sum of weights
print(torch.sum(single_neuron.weight))

tensor(0.5105, grad_fn=<SumBackward0>)


## Linear Layer with 4 input features and 3 output features

In [13]:
linear_layer = nn.Linear(4, 3, bias=True)

In [14]:
linear_input = torch.rand((1, 4))

In [15]:
out_feats = linear_layer(linear_input)
print(out_feats.shape)
linear_layer.weight.shape

torch.Size([1, 3])


torch.Size([3, 4])

In [19]:
linear_layer.bias.shape

torch.Size([3])

A linear layer is applied to an input tensor by applying a linear transformation of the form $y = xA^T + b$. This can be easily varified by evaluating the equation manually:

In [18]:
torch.mm(linear_input, linear_layer.weight.T) + linear_layer.bias

tensor([[-0.3321, -0.3663,  0.4299]], grad_fn=<AddBackward0>)

In [20]:
result = torch.mm(linear_input, linear_layer.weight.T)

In [21]:
result.shape

torch.Size([1, 3])

In [23]:
linear_layer.bias

Parameter containing:
tensor([-0.3696,  0.1267, -0.0788], requires_grad=True)

In [17]:
# easy to produce errors with wrong layout
print( linear_layer.bias.shape)
print( torch.mm(linear_layer.weight, linear_input.T) + linear_layer.bias.T )
print( torch.mm(linear_layer.weight, linear_input.T) + linear_layer.bias)
print( torch.mm(linear_layer.weight, linear_input.T) + linear_layer.bias.reshape(3,1) ) #only working line of code

print( torch.mm(linear_layer.weight, linear_input) + linear_layer.bias)



torch.Size([3])
tensor([[-0.3321,  0.1642, -0.0413],
        [-0.8626, -0.3663, -0.5718],
        [ 0.1390,  0.6354,  0.4299]], grad_fn=<AddBackward0>)
tensor([[-0.3321,  0.1642, -0.0413],
        [-0.8626, -0.3663, -0.5718],
        [ 0.1390,  0.6354,  0.4299]], grad_fn=<AddBackward0>)
tensor([[-0.3321],
        [-0.3663],
        [ 0.4299]], grad_fn=<AddBackward0>)


/tmp/ipykernel_117274/3487998597.py:3: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2985.)
  print( torch.mm(linear_layer.weight, linear_input.T) + linear_layer.bias.T )


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x4 and 1x4)

In [16]:
# numpy verification
result = np.dot(linear_input.detach().numpy(), linear_layer.weight.T.detach().numpy()) +linear_layer.bias.reshape(3).detach().numpy()
print('Result: \n', result.shape)


Result: 
 (1, 3)


## Multi Layer Perceptron (MLP)

A Neural Network consisting of more than one linear layer (also referred to as Fully Connected Layer) is also called a Multi Layer Perceptron (MLP).

In [ ]:
# often a neural network is defined as object derived from "nn.Module"
# we would like to define an "__init__" and "forward" function

class ThreeLayerMLP(nn.Module):
    def __init__(self, input_features, hidden_features, output_features):
        super(ThreeLayerMLP, self).__init__()
        self.layer_0 = nn.Linear(in_features = input_features, out_features = hidden_features)
        self.layer_1 = nn.Linear(in_features = hidden_features, out_features = hidden_features)
        self.layer_2 = nn.Linear(in_features = hidden_features, out_features = output_features)
        
    def forward(self, x):
        x = self.layer_0(x)
        x = self.layer_1(x)
        x = self.layer_2(x)
        
        return x

In [ ]:
# initialization
my_mlp = ThreeLayerMLP(10, 5, 1)

In [ ]:
mlp_input = torch.rand((1, 10))

In [ ]:
my_mlp(mlp_input) # this calls the forward function; handles distribution on multiple GPUs

In [24]:
# add activation functions
# 

class ThreeLayerMLPNew(nn.Module):
    def __init__(self, input_features, hidden_features, output_features):
        super(ThreeLayerMLPNew, self).__init__()
        self.layer_0 = nn.Linear(in_features = input_features, out_features = hidden_features)
        self.layer_1 = nn.Linear(in_features = hidden_features, out_features = hidden_features)
        self.layer_2 = nn.Linear(in_features = hidden_features, out_features = output_features)
        self.relu = nn.ReLU() # only defined once not three times / depends on coding style / better to define layer for each use

    def forward(self, x):
        x = self.layer_0(x)
        x = self.relu(x)
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.layer_2(x)
        x = self.relu(x)
        
        return x

In [27]:
# execute multiple times to see that output never gets < 0

my_mlpnew = ThreeLayerMLPNew(10, 10, 2)
mlp_input = torch.rand((1, 10))
my_mlpnew(mlp_input)

tensor([[0.0000, 0.0410]], grad_fn=<ReluBackward0>)

# Conv2d Layer ([Documentation](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html))
``torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)``

In [ ]:
conv_input = torch.randn(1, 3, 9, 9) # batch_size == 1, num_channels == 3, height == 9, width == 9

In [ ]:
# With square 3x3 kernels
m = nn.Conv2d(3, 1, 3, stride = 1, padding = 1)

In [ ]:
output = m(conv_input)
print(output.shape)
print(output)

In [ ]:
# With square 3x3 kernels and stride 2
m = nn.Conv2d(3, 1, 3, stride =  2, padding = 1)

In [ ]:
output = m(conv_input)
print(output.shape)
print(output)

## Convolutional Neural Network

A Neural Network that only consists of convolutional layers is also referred to as a convolutional neural network. The big advantage is that the input size can vary as it only has to have the right number of input channels but the spatial height and width can vary.

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, input_channels, hidden_channels, output_channels):
        super(ConvNet, self).__init__()
        self.layer_0 = nn.Conv2d(in_channels = input_channels, out_channels = hidden_channels, kernel_size = 3, stride = 1, padding = 1)
        self.relu0 = nn.ReLU()
        self.layer_1 = nn.Conv2d(in_channels = hidden_channels, out_channels = hidden_channels, kernel_size = 3, stride = 1, padding = 1)
        self.relu1 = nn.ReLU()
        self.layer_2 = nn.Conv2d(in_channels = hidden_channels, out_channels = output_channels, kernel_size = 3, stride = 1, padding = 1)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.layer_0(x)
        x = self.relu0( x )
        x = self.layer_1(x)
        x = self.relu1( x )
        x = self.layer_2(x)
        x = self.relu2( x )
        
        return x

# TODO
* Randomly initialise weight
* Implement forward propagation to get ) for any
* Implement backprop to compute partial derivatives
* For all the samples, perform forward propagation and
backpropagation
* Using numerical estimation of gradient to check the gradient
calculation, disable after checking
* Use gradient descent or advanced optimization method with
backpropagation to try to minimize cost function